In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## Load the model
model = load_model("model.keras")

## Load the encoders and the scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
## One hot encode the 'Geography' feature from the input
## This was the reason why we saved the encoder and the scaler

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\vedan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])

In [6]:
# Combine one-hot encoded columns with input data
input_df = pd.concat([input_df.drop("Geography",axis = 1),geo_encoded_df], axis = 1)

In [7]:
## Encode categortical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## Scaling 
input_scaled = scaler.transform(input_df)
input_scaled


array([[-0.55012981,  0.89091075,  0.12136034, -0.68041201, -0.28051905,
         0.84584804,  0.62670381,  0.968496  , -0.91572441,  1.00400803,
        -0.57427105, -0.58350885]])

In [9]:
#Predict churn
prediction = model.predict(input_scaled)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 587ms/step


In [10]:
prediction_probab = prediction[0][0]
prediction_probab

np.float32(0.052612167)

In [11]:
if prediction_probab > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
